In [1]:
from pyniryo import *
import pygame
import numpy as np
import time
import math
import threading
from azure.iot.device import IoTHubDeviceClient, Message
import json
import time

pygame 2.5.1 (SDL 2.28.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# robot_ip_address = "169.254.200.200"
robot_ip_address = "10.10.10.10"
screen_size = (640, 480)
screen_title = 'Niryo Robot Control'

In [3]:
screen = pygame.display.set_mode(screen_size)
pygame.display.set_caption(screen_title)

In [4]:
robot = NiryoRobot(robot_ip_address)


Connected to server (10.10.10.10) on port: 40001



In [8]:
robot.calibrate_auto()

In [9]:
move_speed = 0.25
rot_speed = 0.25

In [10]:
robot.move_to_home_pose()

In [11]:
device_connection_string = "HostName=niryoiot.azure-devices.net;DeviceId=NiryoNed2;SharedAccessKey=Ee0uclVHdESQI+NPmG8z5G3ZA1ET0dkpiZ5lbyOY4Oo=="
device_client = IoTHubDeviceClient.create_from_connection_string(device_connection_string)
device_client.connect()

In [12]:
done = False
joints = []
previous_joints = None
last_sent_time = 0
message_interval = 2
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
    pose = robot.get_pose()
    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        pose.x +=0.045
        robot.move_pose(pose)
        joints = robot.get_joints()
    if keys[pygame.K_s]:
        pose.x -=0.045
        robot.move_pose(pose)
        joints = robot.get_joints()
    if keys[pygame.K_a]:
        pose.y +=0.045
        robot.move_pose(pose)
        joints = robot.get_joints()
    if keys[pygame.K_d]:
        pose.y -=0.045
        robot.move_pose(pose)
        joints = robot.get_joints()
    if keys[pygame.K_SPACE]:
        pose.z +=0.045
        robot.move_pose(pose)
        joints = robot.get_joints()
    if keys[pygame.K_LCTRL]:
        pose.z -=0.045
        robot.move_pose(pose)
        joints = robot.get_joints()
    if keys[pygame.K_q]:
        pose.roll -=0.5
        robot.move_pose(pose)
        joints = robot.get_joints()
    if keys[pygame.K_e]:
        pose.roll +=0.5
        robot.move_pose(pose)
        joints = robot.get_joints()
    if keys[pygame.K_o]:
        robot.open_gripper(speed=500)
        joints = robot.get_joints()
    if keys[pygame.K_c]:
        robot.close_gripper(speed=500)
        joints = robot.get_joints()
    if keys[pygame.K_l]:
        done = True

    current_time = time.time()
    if joints != previous_joints and current_time - last_sent_time > message_interval:
            try:
                message = Message(json.dumps(joints))
                device_client.send_message(message)
                last_sent_time = current_time
            except Exception as e:
                print(f"Error sending message to Azure IoT Hub: {e}")
            previous_joints = joints
    
    screen.fill((255, 255, 255))
    pygame.display.update()

    time.sleep(0.01)

In [5]:
robot.close_connection()
device_client.disconnect()


Disconnected from robot



NameError: name 'device_client' is not defined